In [ ]:
import numpy as np
import pandas as pd
import requests
import json

_API key below_

In [ ]:
sg_api = '<your SafeGraph Place API>'

# a function converts table to DataFrame

In [ ]:
import arcpy
def table_to_df(layer):
    fields = []
    for field in arcpy.ListFields(layer):
        fields.append(field.name)
    arr = arcpy.da.TableToNumPyArray(layer, fields[2:])
    return pd.DataFrame(arr, columns=arr.dtype.names)

In [ ]:
df = table_to_df('cp_dc_noduplicate_latlon')
df.head()

## Read cultural places (identifiable only) in DC

In [ ]:
csv_path = r"./cultural_resrouce_dc.csv"
df = pd.read_csv(csv_path)
df.head()

## Lookup by name and postal code

In [ ]:
def address_to_pk(loc_name, address, zipcode):
    headers = {
        'apikey': cc_sg_api,
        'content-type': 'application/json',
    }

    location_name = loc_name
    street_address = address  # street_address = '15th St NW, Washington'
    postal_code = zipcode  # postal_code = '20050'
    iso_country_code = 'US'

    cols = [
        'location_name',
        'street_address',
        'city',
        'region',
        'iso_country_code',
        'postal_code'
    ]
    cols = ' '.join(cols)

    lookup = '''
        lookup(query: {
        location_name: \\"%s\\",
        street_address: \\"%s\\",
        postal_code: \\"%s\\",
        iso_country_code: \\"%s\\"})
    '''
    lookup = lookup % (location_name, street_address, postal_code, iso_country_code)
    lookup = lookup.strip().replace('\n', '')

    data = '{"query": "query {%s {placekey safegraph_core {%s}}}", "variables":{}}'
    response = requests.post('https://api.safegraph.com/v1/graphql',
                             headers=headers,
                             data=data % (lookup, cols))
    
    sg_records_num = json.loads("".join(response.text))['extensions']['row_count']
    if sg_records_num > 0:
        print(f'{sg_records_num} sg place found.')
        return json.loads("".join(response.text))['data']['lookup']['placekey']
    else:
        print('No record found.')

In [ ]:
x = address_to_pk('Washington Monument', '2483 Tilden St NW', '20008') # google reverse geocoding address no matching

## Lookup by name and lat & lon

In [ ]:
def lat_lon_pk(loc_name, lat, lon):
    headers = {
        'apikey': cc_sg_api,
        'content-type': 'application/json',
    }

    location_name = loc_name
    latitude = str(lat)
    longitude = str(lon)
    iso_country_code = 'US'

    cols = [
        'location_name',
        'street_address',
        'latitude',
        'longitude',
        'postal_code'
    ]
    cols = ' '.join(cols)

    lookup = '''
        lookup(query: {
        location_name: \\"%s\\",
        latitude: %s,
        longitude: %s,
        iso_country_code: \\"%s\\"})
    '''
    lookup = lookup % (location_name, latitude, longitude, iso_country_code)
    lookup = lookup.strip().replace('\n', '')

    data = '{"query": "query {%s {placekey safegraph_core {%s}}}", "variables":{}}'
    response = requests.post('https://api.safegraph.com/v1/graphql',
                             headers=headers,
                             data=data % (lookup, cols))
    sg_records_num = json.loads("".join(response.text))['extensions']['row_count']
    if sg_records_num > 0:
        print(f'{sg_records_num} sg place found.')
        return (json.loads("".join(response.text))['data']['lookup']['placekey'],
                json.loads("".join(response.text))['data']['lookup']['safegraph_core']['latitude'],
                json.loads("".join(response.text))['data']['lookup']['safegraph_core']['longitude'])
    else:
        print('No record found.')
        return np.nan, np.nan, np.nan

In [ ]:
lat_lon_pk('Washington Monument', 38.889484, -77.035279)   # able to retrieve using lat lon whereas google API address couldn't

## Lookup by placekey

In [ ]:
headers = {
    'apikey': cc_sg_api,
    'content-type': 'application/json',
}

placekey = 'zzw-222@63r-6mx-wkz'
lookup = 'lookup(placekey: \\"{}\\")'.format(placekey)
cols = [
    'placekey',
    'latitude',
    'longitude',
    'street_address',
    'location_name'
]
cols = ' '.join(cols)
data = '{"query": "query {%s {safegraph_core {%s}}}", "variables":{}}'
response = requests.post('https://api.safegraph.com/v1/graphql',
                         headers=headers,
                         data=data % (lookup, cols))

In [ ]:
json.loads("".join(response.text))['data']['lookup']['safegraph_core']['location_name']

In [ ]:
lat_lon_pk('RunningAsana', 38.94076566100159, -77.05212427944639)

## Verify how well matches SafeGraph with lat lon

In [ ]:
df['sg_pk'], df['sg_lat'], df['sg_lon'] = zip(*df.apply(lambda x: lat_lon_pk(x['Item'], x['lat'], x['lon']), axis=1))

In [ ]:
print(f"{df['sg_pk'].count()} out of {len(df)} records found matching records in SafeGraph")

In [ ]:
df.head()

In [ ]:
csv_path = "./cultural_rescoure_dc_sg.csv" 
df.to_csv(csv_path, index=False)